In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
from torch.optim import SGD
from torch import nn

## Access src directory from ./notebooks/ folder
from pathlib import Path
sys.path.append(str(Path('.').absolute().parent))

from src.Snip import SNIP
from src.models import ConvNet
from src.optimization import epoch

import numpy as np

### Manual test with F-MNIST

In [ ]:
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms

# Prepare dataloaders

In [ ]:
pruning_batch_size = 128
batch_size = 64
path = '../data/'
data_transforms = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = FashionMNIST(path, train=True, download=True,transform=data_transforms)
test_dataset = FashionMNIST(path, train=False, download=True,transform=data_transforms)
pruning_data_loader = torch.utils.data.DataLoader(train_dataset,
                        batch_size=pruning_batch_size, shuffle=True, num_workers=2)

train_data_loader = torch.utils.data.DataLoader(train_dataset,
                        batch_size=batch_size, shuffle=True, num_workers=2)
test_data_loader = torch.utils.data.DataLoader(test_dataset,
                        batch_size=batch_size, shuffle=True, num_workers=2)

# Execute pruning
For given model, execute SNIP pruning using one batch of 128 examples. K is the number of connections to keep.

In [ ]:
model = ConvNet()
snip = SNIP(model)
C, mapping = snip.compute_mask(pruning_data_loader, K=1000)

# Train the model
In the beginning of each batch, use pruning to force the connections to zero.

In [ ]:
epochs = 30
#lr = 1e-5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
#lr_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

model = model.cuda()
criterion = criterion.cuda()

# On itère sur les epochs
for i in range(epochs):
    print("=================\n=== EPOCH "+str(i+1)+" =====\n=================\n")
    # Phase de train
    acc, loss = epoch(train_data_loader, model, criterion,snip_pruning=snip, optimizer=optimizer)
    # Phase d'evaluation
    with torch.no_grad():
        acc_test, loss_test = epoch(test_data_loader, model, criterion)
    # plot
    #plot.update(loss.avg, loss_test.avg, top1_acc.avg, top1_acc_test.avg)